In [1]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    average_precision_score, roc_auc_score, precision_recall_curve,
    f1_score, precision_score, recall_score, confusion_matrix, classification_report
)
import time
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from xgboost import XGBClassifier
import shap
import os
from datetime import datetime
from joblib import dump, load
import warnings
from sklearn.exceptions import FitFailedWarning
import xgboost as xgb
import optuna

c:\Users\ayxbhg\.venvs\py313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def read_from_azure_in_chunks(chunksize=100000):
    try:
        # Set up connection
        cnxn = pyodbc.connect(
            'Driver={ODBC Driver 17 for SQL Server};'
            'Server=bhgazuresql01.database.windows.net;'
            'Authentication=ActiveDirectoryPassword;'
            'Database=BHG_DR;'
            'UID=xxxxxxxxxx;'
            'PWD=xxxxxxxxxx;'
        )
        print("Connection successful!")

        # Define SQL query
        sql_query = """
        SELECT
            *
        FROM
            pats.vw_ClaimsDenialPrediction
        WHERE
            ClaimBillDate >= dateadd(year,-1,cast(getdate() as date)) AND LIATPCLIid IS NOT NULL
        """

        # Load data in chunks and concatenate
        chunk_generator = pd.read_sql(sql_query, cnxn, chunksize=chunksize)
        chunks = []
        for i, chunk in enumerate(chunk_generator):
            print(f"Loaded chunk {i+1} with {len(chunk)} rows")
            chunks.append(chunk)

        full_df = pd.concat(chunks, ignore_index=True)
        print(f"\n Total rows combined: {len(full_df)}")
        return full_df

    except pyodbc.DatabaseError as e:
        print('Database Error:', str(e))
    except pyodbc.Error as e:
        print('Connection Error:', str(e))
    finally:
        try:
            cnxn.close()
        except:
            pass

# Main execution
if __name__ == "__main__":
    df = read_from_azure_in_chunks()

    if df is not None:
        print(f"\nTotal rows returned: {len(df)}\n")
        print("Top 5 rows of the DataFrame:")
        print(df.head())
    else:
        print("No data returned.")

Connection successful!


C:\Users\ayxbhg\AppData\Local\Temp\ipykernel_61028\2961411225.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_generator = pd.read_sql(sql_query, cnxn, chunksize=chunksize)


Loaded chunk 1 with 100000 rows
Loaded chunk 2 with 100000 rows
Loaded chunk 3 with 100000 rows
Loaded chunk 4 with 100000 rows
Loaded chunk 5 with 100000 rows
Loaded chunk 6 with 100000 rows
Loaded chunk 7 with 100000 rows
Loaded chunk 8 with 100000 rows
Loaded chunk 9 with 100000 rows
Loaded chunk 10 with 100000 rows
Loaded chunk 11 with 100000 rows
Loaded chunk 12 with 100000 rows
Loaded chunk 13 with 100000 rows
Loaded chunk 14 with 100000 rows
Loaded chunk 15 with 100000 rows
Loaded chunk 16 with 100000 rows
Loaded chunk 17 with 100000 rows
Loaded chunk 18 with 100000 rows
Loaded chunk 19 with 100000 rows
Loaded chunk 20 with 100000 rows
Loaded chunk 21 with 100000 rows
Loaded chunk 22 with 100000 rows
Loaded chunk 23 with 100000 rows
Loaded chunk 24 with 100000 rows
Loaded chunk 25 with 96674 rows

 Total rows combined: 2496674

Total rows returned: 2496674

Top 5 rows of the DataFrame:
  Clinic  TPCLIID  LIATPCLIid   ServiceDt                     Service  \
0    AHK    10541    

In [3]:
print(f"\nTotal rows before deduplication: {len(df)}")

duplicate_count = df.duplicated().sum()
if duplicate_count == 0:
    print("No duplication detected.")
else:
    print(f"Number of duplicated rows: {duplicate_count}")
    df = df.drop_duplicates()
    print(f"Total rows after deduplication: {len(df)}\n")



Total rows before deduplication: 2496674
No duplication detected.


In [4]:
print(df.isnull().sum()) 

Clinic                           0
TPCLIID                          0
LIATPCLIid                       0
ServiceDt                        0
Service                          0
ClaimID                          0
AmountCharged                    0
CPTCode                          0
ClientID                         0
ClaimBillDate                    0
Payer                            0
Provider                         0
AuthStatus                       0
eligStatus                       0
DenialFlag                       0
lastActDt                        0
cliANSI1                   1617902
cliANSI2                   2379914
TotalPaid                        0
TotalAdj                         0
TotalVoid                        0
CoPay                            0
Deduc                            0
CoIns                            0
CltResp                          0
Balance                          0
MultiFlag                        0
SameDayCli                       0
DaysBetServiceToBill

In [5]:
# -*- coding: utf-8 -*-
"""
Full pipeline (native categoricals, random UNSEEN sampling, Optuna tuning, SHAP):
- Keep `tpccltID` (do NOT drop it).
- Use XGBoost **native categoricals** (no one-hot) with `enable_categorical=True`.
- Build **UNSEEN** by random sampling from the remainder after taking the training sample (no date cutoff).
- Engineer compact **date features** and drop raw datetime columns before modeling.
- Optuna tunes on CV PR-AUC (wider search for more trial diversity); choose threshold from OOF.
- Compare **F1/precision/recall** at each trial's OOF-chosen threshold on TEST and UNSEEN.
- Save artifacts; score UNSEEN and Azure data; export **SHAP Top-K** explanations.
"""


# ======================================
# CLEANUP + PREP
# ======================================

# Drop unnecessary columns
cols_to_drop = [
    'cliANSI1', 'cliANSI2', 'TotalPaid', 'TotalAdj', 'TotalVoid','MultiFlag',
    'CoPay', 'Deduc', 'CoIns', 'CltResp', 'ClientResp.', 'Balance', 'tpcliAge'
]
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print(f"Columns dropped: {cols_to_drop}")

# Drop rows with nulls in important columns
before_dropna = len(df)
df = df.dropna(subset=['ServiceDt', 'Service', 'CPTCode'])
after_dropna = len(df)
print(f"Rows dropped due to NULLs : {before_dropna - after_dropna}")
print(f"Total rows after cleaning: {len(df)}")

# Step 1: Drop ID columns (KEEP tpccltID)
df = df.drop(columns=['TPCLIID', 'ClaimID', 'LIATPCLIid'], errors='ignore')

# Step 2: Target type
df['DenialFlag'] = df['DenialFlag'].astype('category')

# Ensure datetime dtype for feature engineering
for c in ['ClaimBillDate', 'ServiceDt', 'lastActDt']:
    df[c] = pd.to_datetime(df[c], errors='coerce')

# -------------------------------
# Helper: Date feature engineering (drop raw datetimes)
# -------------------------------

def add_date_features(df_in: pd.DataFrame) -> pd.DataFrame:
    dfX = df_in.copy()
    # Ensure datetime
    for c in ['ServiceDt', 'ClaimBillDate', 'lastActDt']:
        if c in dfX.columns:
            dfX[c] = pd.to_datetime(dfX[c], errors='coerce')

    # Gaps & recency
    if 'ServiceDt' in dfX.columns and 'ClaimBillDate' in dfX.columns:
        dfX['days_claim_from_service'] = (dfX['ClaimBillDate'] - dfX['ServiceDt']).dt.days
    if 'lastActDt' in dfX.columns and 'ClaimBillDate' in dfX.columns:
        dfX['days_claim_from_lastAct'] = (dfX['ClaimBillDate'] - dfX['lastActDt']).dt.days
    if 'ServiceDt' in dfX.columns:
        min_service = dfX['ServiceDt'].min()
        dfX['service_age_days'] = (dfX['ServiceDt'] - min_service).dt.days

    # Calendar parts
    if 'ServiceDt' in dfX.columns:
        dfX['service_dow'] = dfX['ServiceDt'].dt.weekday.astype('category')  # 0..6
    if 'ClaimBillDate' in dfX.columns:
        dfX['claim_month'] = dfX['ClaimBillDate'].dt.month.astype('category')  # 1..12
        dfX['claim_week']  = dfX['ClaimBillDate'].dt.isocalendar().week.astype('int32')

    # Clip extreme gaps
    for c in ['days_claim_from_service', 'days_claim_from_lastAct', 'service_age_days']:
        if c in dfX.columns:
            dfX[c] = dfX[c].clip(-3650, 36500)

    # Drop raw datetime columns
    dfX = dfX.drop(columns=['ServiceDt', 'ClaimBillDate', 'lastActDt'], errors='ignore')
    return dfX

# --------------------------------------
# Build TRAIN sample and random UNSEEN
# --------------------------------------

# Use full df as pool
df_train_pool = df.copy()

# --- Randomly sample up to 1,000,000 rows from training pool ---
n_sample = min(1_000_000, len(df_train_pool))
df_sample = df_train_pool.sample(n=n_sample, random_state=123)
print("Sampled training rows:", df_sample.shape[0])

# --- UNSEEN: sample from remainder (no date logic) ---
remainder = df_train_pool.drop(index=df_sample.index)
n_unseen = min(10_000, len(remainder))
df_unseen = remainder.sample(n=n_unseen, random_state=456).copy()
print("Unseen (sampled) rows:", df_unseen.shape[0])


Columns dropped: ['cliANSI1', 'cliANSI2', 'TotalPaid', 'TotalAdj', 'TotalVoid', 'MultiFlag', 'CoPay', 'Deduc', 'CoIns', 'CltResp', 'ClientResp.', 'Balance', 'tpcliAge']
Rows dropped due to NULLs : 0
Total rows after cleaning: 2496674
Sampled training rows: 1000000
Unseen (sampled) rows: 10000


In [6]:

# (Optional) QC: NaN DenialFlag by ClaimBillDate on the *original* df before dates are dropped
try:
    nan_counts = (
        df.loc[df['DenialFlag'].isna()]
          .groupby(df['ClaimBillDate'].dt.date)
          .size()
          .reset_index(name='NaN_DenialFlag_Count')
          .sort_values('NaN_DenialFlag_Count', ascending=False)
    )
    print("NaN counts in DenialFlag by ClaimBillDate (overall):")
    print(nan_counts.head(20))
except Exception as _e:
    print("[Info] Skipped NaN-by-date QC:", _e)


NaN counts in DenialFlag by ClaimBillDate (overall):
Empty DataFrame
Columns: [ClaimBillDate, NaN_DenialFlag_Count]
Index: []


In [7]:

# --- Drop raw datetime columns (no date features) ---
drop_dt = ['ServiceDt', 'ClaimBillDate', 'lastActDt']
df_sample = df_sample.drop(columns=drop_dt, errors='ignore')
df_unseen = df_unseen.drop(columns=drop_dt, errors='ignore')


# ======================================
# Model matrix (native categorical)
# ======================================

# Identify categorical predictor columns (exclude target)
categorical_cols = [
    c for c in df_sample.select_dtypes(include=['object', 'category']).columns.tolist()
    if c != 'DenialFlag'
]
# Ensure `tpccltID` included as categorical if present
if 'tpccltID' in df_sample.columns and 'tpccltID' not in categorical_cols:
    categorical_cols.append('tpccltID')

# Cast to pandas 'category'
for c in categorical_cols:
    if c in df_sample.columns:
        df_sample[c] = df_sample[c].astype('category')
    if c in df_unseen.columns:
        df_unseen[c] = df_unseen[c].astype('category')

# Build modeling frame (drop NA target)
df_model = df_sample[df_sample['DenialFlag'].notna()].copy()

# X / y
y = df_model['DenialFlag'].astype(int)
X = df_model.drop(columns=['DenialFlag'])

# Lock categorical dtypes from X
cat_dtypes = {c: X[c].dtype for c in categorical_cols if c in X.columns}

# --- Split: 20% test, then 25% of remaining as val (=> 56/14/30 overall) ---
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.20, random_state=123, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=123, stratify=y_train_full
)

# Re-apply categorical dtypes consistently
for c, dt in cat_dtypes.items():
    if c in X_train: X_train[c] = X_train[c].astype(dt)
    if c in X_val:   X_val[c]   = X_val[c].astype(dt)
    if c in X_test:  X_test[c]  = X_test[c].astype(dt)

print("Train sample shape:", df_sample.shape)
print("X_train:", X_train.shape, "X_val:", X_val.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_val:", y_val.shape, "y_test:", y_test.shape)
print("Ratio of 1 in y_train:", y_train.mean())


Train sample shape: (1000000, 12)
X_train: (600000, 11) X_val: (200000, 11) X_test: (200000, 11)
y_train: (600000,) y_val: (200000,) y_test: (200000,)
Ratio of 1 in y_train: 0.35206166666666666


In [8]:
# X / y
y = df_model['DenialFlag'].astype(int)
X = df_model.drop(columns=['DenialFlag'])

# === NEW: Inspect and record predictor columns used for training ===
feature_cols = X.columns.tolist()

# Sanity check: make sure dropped datetime columns didn't sneak back in
leaked_dt = set(drop_dt).intersection(feature_cols)
assert not leaked_dt, f"Unexpected datetime columns in features: {leaked_dt}"

# Helpful breakdown
cat_in_X = [c for c in categorical_cols if c in feature_cols]
num_in_X = [c for c in feature_cols if c not in cat_in_X]

print(f"[Predictors] Using {len(feature_cols)} columns for training.")
print("• Feature columns:", feature_cols)
print(f"• Categorical ({len(cat_in_X)}): {cat_in_X}")
print(f"• Numeric/Other ({len(num_in_X)}): {num_in_X}")

# (Optional) Persist the feature list for future runs/audits
# pd.Series(feature_cols, name="feature").to_csv("training_feature_columns.csv", index=False)


[Predictors] Using 11 columns for training.
• Feature columns: ['Clinic', 'Service', 'AmountCharged', 'CPTCode', 'ClientID', 'Payer', 'Provider', 'AuthStatus', 'eligStatus', 'SameDayCli', 'DaysBetServiceToBilling']
• Categorical (7): ['Clinic', 'Service', 'CPTCode', 'Payer', 'Provider', 'AuthStatus', 'eligStatus']
• Numeric/Other (4): ['AmountCharged', 'ClientID', 'SameDayCli', 'DaysBetServiceToBilling']


In [9]:

# ======================================
# Helpers
# ======================================

def pick_threshold_max_f1(y_true, proba, grid=None):
    if grid is None:
        grid = np.linspace(0.01, 0.99, 99)
    best = (-1.0, 0.5, 0.0, 0.0)  # f1, thr, p, r
    for thr in grid:
        pred = (proba >= thr).astype(int)
        p = precision_score(y_true, pred, zero_division=0)
        r = recall_score(y_true, pred, zero_division=0)
        f1 = f1_score(y_true, pred, zero_division=0)
        if f1 > best[0]:
            best = (f1, float(thr), float(p), float(r))
    return best[1], best[0], best[2], best[3]


def evaluate_at_threshold(y_true, proba, threshold=0.5):
    pred = (proba >= threshold).astype(int)
    cm = confusion_matrix(y_true, pred, labels=[0, 1])
    metrics = {
        "precision": precision_score(y_true, pred, zero_division=0),
        "recall": recall_score(y_true, pred, zero_division=0),
        "f1": f1_score(y_true, pred, zero_division=0),
        "pr_auc": average_precision_score(y_true, proba),
    }
    clsrep = classification_report(y_true, pred, digits=4)
    return cm, metrics, clsrep


def print_results(model_name, cm, metrics, clsrep):
    print(f"=== {model_name} RESULTS ===")
    print("Confusion Matrix [rows=true 0/1, cols=pred 0/1]:", cm)
    print("Metrics:")
    for k, v in metrics.items():
        print(f"  {k}: {v:.5f}")
    print("Classification Report:", clsrep)


In [10]:

# ======================================
# Assemble train+val, imbalance weight
# ======================================
X_trval = pd.concat([X_train, X_val], axis=0)
Y_trval = pd.concat([y_train, y_val], axis=0)

# Ensure cat dtypes on X_trval
for c, dt in cat_dtypes.items():
    if c in X_trval: X_trval[c] = X_trval[c].astype(dt)

pos = int(np.sum((y_train == 1).astype(int)))
neg = int(np.sum((y_train == 0).astype(int)))
scale_pos_weight = float(neg / max(pos, 1))
print(f"scale_pos_weight (initial) = {scale_pos_weight:.3f}")


scale_pos_weight (initial) = 1.840


In [11]:

# ======================================
# CV & Optuna objective (wider search)
# ======================================
CV = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

def split_fn():
    return CV.split(X_trval, Y_trval)


def objective(trial):
    params = {
        "tree_method": "hist",
        "objective": "binary:logistic",
        "eval_metric": "aucpr",  # keep for early stopping; we're optimizing F1 ourselves
        "eta": trial.suggest_float("eta", 0.02, 0.20, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0, log=True),
        "subsample": trial.suggest_categorical("subsample", [0.5, 0.7, 0.85, 1.0]),
        "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.5, 0.7, 0.85, 1.0]),
        "lambda": trial.suggest_float("reg_lambda", 1e-6, 50.0, log=True),
        "alpha":  trial.suggest_float("reg_alpha",  1e-6, 20.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_bin": trial.suggest_categorical("max_bin", [64, 128, 256]),
        "max_cat_to_onehot": trial.suggest_categorical("max_cat_to_onehot", [1, 2, 10]),
        "sampling_method": trial.suggest_categorical("sampling_method", ["uniform"]),  # CPU-safe
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.25*scale_pos_weight, 4.0*scale_pos_weight),
        "seed": 123,
    }

    f1s = []
    for tr_idx, va_idx in split_fn():
        dtr = xgb.DMatrix(X_trval.iloc[tr_idx], label=Y_trval.iloc[tr_idx], enable_categorical=True)
        dva = xgb.DMatrix(X_trval.iloc[va_idx], label=Y_trval.iloc[va_idx], enable_categorical=True)

        booster = xgb.train(
            params,
            dtr,
            num_boost_round=4000,
            evals=[(dva, "val")],
            early_stopping_rounds=150,
            verbose_eval=False,
        )

        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        # pick best threshold by F1 on the validation fold (class 1 focus)
        thr, f1_fold, p_fold, r_fold = pick_threshold_max_f1(Y_trval.iloc[va_idx], proba)
        f1s.append(float(f1_fold))

    # Optuna will MAXIMIZE this mean F1
    return float(np.mean(f1s))


# Run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=40, show_progress_bar=False)
print("Best trial:")
print("  value (mean CV PR-AUC):", study.best_value)
print("  params:")
for k, v in study.best_params.items():
    print(f"    {k}: {v}")


[I 2025-09-09 17:36:12,567] A new study created in memory with name: no-name-d4710e2d-4aa4-454a-9021-7553fe2bcbdd
[I 2025-09-09 17:48:55,833] Trial 0 finished with value: 0.892223394268213 and parameters: {'eta': 0.16282276974018148, 'max_depth': 4, 'min_child_weight': 5.966661742985204, 'subsample': 0.85, 'colsample_bytree': 0.7, 'reg_lambda': 1.4138305642553758e-06, 'reg_alpha': 7.254507810340025e-06, 'gamma': 3.7997552301502155, 'grow_policy': 'lossguide', 'max_bin': 64, 'max_cat_to_onehot': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 2.532254762890173}. Best is trial 0 with value: 0.892223394268213.
[I 2025-09-09 17:56:53,403] Trial 1 finished with value: 0.901327056753425 and parameters: {'eta': 0.12772060229585225, 'max_depth': 12, 'min_child_weight': 1.6323309366572076, 'subsample': 0.85, 'colsample_bytree': 1.0, 'reg_lambda': 1.0100791325912743e-06, 'reg_alpha': 0.11936197780746717, 'gamma': 4.742139397500581, 'grow_policy': 'depthwise', 'max_bin': 256, 'max_cat_to_one

Best trial:
  value (mean CV PR-AUC): 0.9031766516531127
  params:
    eta: 0.07690013575165573
    max_depth: 11
    min_child_weight: 1.399856728745255
    subsample: 0.85
    colsample_bytree: 0.5
    reg_lambda: 0.00046823259526695716
    reg_alpha: 0.003141640187603369
    gamma: 1.870395577060695
    grow_policy: depthwise
    max_bin: 256
    max_cat_to_onehot: 2
    sampling_method: uniform
    scale_pos_weight: 3.9351479278590102


In [12]:
# ----------------------------------------------
# Evaluate top-K trials (refit + TEST/UNSEEN under precision constraint)
# ----------------------------------------------

from pandas.api.types import CategoricalDtype as _CatDType
from sklearn.metrics import precision_recall_curve, average_precision_score

# --- Helper: smallest threshold that meets precision_floor (maximizes recall under the constraint)
def threshold_for_precision(y_true, proba, precision_floor=0.90):
    p, r, thr = precision_recall_curve(y_true, proba)
    # Walk thresholds in ascending order; pick the smallest meeting the floor
    for P, T in zip(p[:-1], thr):  # last precision has no corresponding threshold
        if P >= precision_floor:
            return float(T)
    # If nothing meets the floor, fall back to the largest available threshold (max precision)
    return float(thr[-1]) if len(thr) else 0.5

# --- OOF: pick threshold to maximize recall with precision >= floor; also get median best n_estimators
def get_oof_threshold_and_n(X_ref, y_ref, params, precision_floor=0.90):
    oof_proba = np.full(shape=(len(y_ref),), fill_value=np.nan, dtype=float)
    best_iterations = []
    for tr_idx, va_idx in split_fn():
        dtr = xgb.DMatrix(X_ref.iloc[tr_idx], label=y_ref.iloc[tr_idx], enable_categorical=True)
        dva = xgb.DMatrix(X_ref.iloc[va_idx], label=y_ref.iloc[va_idx], enable_categorical=True)
        booster = xgb.train(
            params,
            dtr,
            num_boost_round=4000,
            evals=[(dva, "val")],
            early_stopping_rounds=150,
            verbose_eval=False,
        )
        proba = booster.predict(dva, iteration_range=(0, booster.best_iteration + 1))
        oof_proba[va_idx] = proba
        best_iterations.append(int(booster.best_iteration) + 1)

    assert np.isfinite(oof_proba).all(), "OOF predictions contain NaNs"

    # Choose operating point: smallest threshold with precision >= floor (thus highest recall under the constraint)
    thr = threshold_for_precision(y_ref, oof_proba, precision_floor=precision_floor)

    # Keep same return signature/names
    n_estimators = int(np.median(best_iterations))
    pr_oof = float(average_precision_score(y_ref, oof_proba))
    return float(thr), n_estimators, pr_oof

# Prepare UNSEEN target and alignment helper
if 'DenialFlag' in df_unseen.columns:
    try:
        y_unseen = df_unseen['DenialFlag'].astype(int)
    except Exception:
        y_unseen = None
else:
    y_unseen = None

# Lock categorical category lists for alignment
cat_dtype_map = {c: list(cat_dtypes[c].categories) for c in cat_dtypes}
feature_cols = X.columns.tolist()

def align_to_training_native(df_raw: pd.DataFrame, feature_cols: list, cat_dtype_map: dict) -> pd.DataFrame:
    dfX = df_raw.copy()
    # Drop target if present
    if 'DenialFlag' in dfX.columns:
        dfX = dfX.drop(columns=['DenialFlag'])
    # Add any missing columns
    for c in feature_cols:
        if c not in dfX.columns:
            if c in cat_dtype_map:
                dfX[c] = pd.Series(pd.Categorical([None]*len(dfX), dtype=_CatDType(cat_dtype_map[c])))
            else:
                dfX[c] = 0
    # Enforce categorical dtypes with locked categories
    for c, cats in cat_dtype_map.items():
        if c in dfX.columns:
            dfX[c] = dfX[c].astype(_CatDType(cats))
    # Reorder
    return dfX[feature_cols]

# Build UNSEEN features aligned to TRAIN
X_unseen = align_to_training_native(df_unseen, feature_cols, cat_dtype_map)

# Select top-K trials by CV PR-AUC
top_k = 3
ranked_trials = sorted(
    study.trials,
    key=lambda t: t.value if t.value is not None else -np.inf,
    reverse=True
)[:top_k]

comparison_rows = []
final_artifacts = None

# You can tweak this floor if needed; default keeps signature compatibility
_precision_floor = 0.90

for rank, tr in enumerate(ranked_trials, start=1):
    params = tr.params.copy()
    params.update({
        "tree_method": "hist",
        "objective": "binary:logistic",
        "eval_metric": "aucpr",
        "seed": 123,
    })

    # OOF-based operating point that maximizes recall under precision >= floor
    thr_i, n_i, pr_oof_i = get_oof_threshold_and_n(X_trval, Y_trval, params, precision_floor=_precision_floor)

    xgb_refit = XGBClassifier(
        tree_method="hist",
        objective="binary:logistic",
        eval_metric=None,
        n_estimators=n_i,
        learning_rate=params.get("eta", 0.05),
        max_depth=params.get("max_depth", 8),
        min_child_weight=params.get("min_child_weight", 2),
        subsample=params.get("subsample", 0.9),
        colsample_bytree=params.get("colsample_bytree", 0.8),
        reg_lambda=params.get("lambda", 1.0),
        reg_alpha=params.get("alpha", 0.0),
        gamma=params.get("gamma", 0.0),
        grow_policy=params.get("grow_policy", "depthwise"),
        max_bin=params.get("max_bin", 256),
        max_cat_to_onehot=params.get("max_cat_to_onehot", 1),
        sampling_method=params.get("sampling_method", "uniform"),
        random_state=123,
        n_jobs=-1,
        scale_pos_weight=params.get("scale_pos_weight", scale_pos_weight),
        enable_categorical=True,
    )

    # Ensure categorical dtype for refit inputs
    for c, dt in cat_dtypes.items():
        if c in X_trval: X_trval[c] = X_trval[c].astype(dt)
        if c in X_test:  X_test[c]  = X_test[c].astype(dt)

    xgb_refit.fit(X_trval, Y_trval)

    # TEST eval at the recall-maximizing (precision-floor) threshold
    proba_test = xgb_refit.predict_proba(X_test)[:, 1]
    cm_test, metrics_test, _ = evaluate_at_threshold(y_test, proba_test, threshold=thr_i)

    # UNSEEN eval (if labels exist)
    unseen_summary = {"f1": np.nan, "precision": np.nan, "recall": np.nan, "pr_auc": np.nan}
    if y_unseen is not None:
        dfx_u = X_unseen.copy()
        for c, cats in cat_dtype_map.items():
            if c in dfx_u.columns:
                dfx_u[c] = dfx_u[c].astype(_CatDType(cats))
        proba_unseen = xgb_refit.predict_proba(dfx_u)[:, 1]
        _, metrics_unseen, _ = evaluate_at_threshold(y_unseen, proba_unseen, threshold=thr_i)
        unseen_summary = metrics_unseen

    comparison_rows.append({
        "rank": rank,
        "trial": tr.number,
        "mean_cv_pr_auc": float(tr.value) if tr.value is not None else np.nan,
        "n_estimators": n_i,
        "thr": float(thr_i),
        "TEST_f1": float(metrics_test["f1"]),
        "TEST_precision": float(metrics_test["precision"]),
        "TEST_recall": float(metrics_test["recall"]),
        "TEST_pr_auc": float(metrics_test["pr_auc"]),
        "UNSEEN_f1": float(unseen_summary["f1"]) if not np.isnan(unseen_summary["f1"]) else np.nan,
        "UNSEEN_precision": float(unseen_summary["precision"]) if not np.isnan(unseen_summary["precision"]) else np.nan,
        "UNSEEN_recall": float(unseen_summary["recall"]) if not np.isnan(unseen_summary["recall"]) else np.nan,
        "UNSEEN_pr_auc": float(unseen_summary["pr_auc"]) if not np.isnan(unseen_summary["pr_auc"]) else np.nan,
    })

    # Save the best-by-CV artifacts (first iteration is top-ranked)
    if rank == 1:
        # Keep key names the same
        final_artifacts = {
            "model": xgb_refit,
            "feature_cols": feature_cols,
            "threshold": float(thr_i),  # precision-floor operating point
            "categorical_cols": categorical_cols,
            "cat_dtypes": cat_dtype_map,
            "params": params,
        }
        # Print detailed TEST/UNSEEN report for the top model
        cmT, metT, repT = evaluate_at_threshold(y_test, proba_test, threshold=thr_i)
        print_results("Top Trial — TEST (precision floor)", cmT, metT, repT)
        if y_unseen is not None:
            cmU, metU, repU = evaluate_at_threshold(y_unseen, proba_unseen, threshold=thr_i)
            print_results("Top Trial — UNSEEN (precision floor)", cmU, metU, repU)

# Show comparison across the top trials
comp_df = pd.DataFrame(comparison_rows)
print("=== Top Trials Comparison (metrics at each trial's precision-floor threshold) ===")
print(comp_df.to_string(index=False))

# Persist artifacts for the best-by-CV model
if final_artifacts is not None:
    dump(final_artifacts, "xgb_claims_binmodel_nativecat.joblib")
    print("Saved: xgb_claims_binmodel_nativecat.joblib")


=== Top Trial — TEST (precision floor) RESULTS ===
Confusion Matrix [rows=true 0/1, cols=pred 0/1]: [[122641   6946]
 [  6831  63582]]
Metrics:
  precision: 0.90151
  recall: 0.90299
  f1: 0.90225
  pr_auc: 0.97290
Classification Report:               precision    recall  f1-score   support

           0     0.9472    0.9464    0.9468    129587
           1     0.9015    0.9030    0.9022     70413

    accuracy                         0.9311    200000
   macro avg     0.9244    0.9247    0.9245    200000
weighted avg     0.9311    0.9311    0.9311    200000

=== Top Trial — UNSEEN (precision floor) RESULTS ===
Confusion Matrix [rows=true 0/1, cols=pred 0/1]: [[6142  327]
 [ 310 3221]]
Metrics:
  precision: 0.90784
  recall: 0.91221
  f1: 0.91002
  pr_auc: 0.97371
Classification Report:               precision    recall  f1-score   support

           0     0.9520    0.9495    0.9507      6469
           1     0.9078    0.9122    0.9100      3531

    accuracy                         0.

In [13]:

# ============================
# PREDICT ON df_unseen (scored)
# ============================

# Score + classify with saved artifacts (already in memory)
model = final_artifacts["model"]
feature_cols = final_artifacts["feature_cols"]
thr = float(final_artifacts["threshold"])
cat_dtype_map = final_artifacts["cat_dtypes"]

# Build X for the UNSEEN set (already engineered dates)
X_unseen_for_scoring = align_to_training_native(df_unseen, feature_cols, cat_dtype_map)
proba_unseen = model.predict_proba(X_unseen_for_scoring)[:, 1]
pred_unseen  = (proba_unseen >= thr).astype(int)

# Package results (keep ALL df_unseen columns)
results_unseen = df_unseen.copy()
results_unseen["xgb_proba"] = proba_unseen
results_unseen["xgb_pred"]  = pred_unseen

# Attach true labels if available
if y_unseen is not None:
    results_unseen["DenialFlag_true"] = y_unseen.values

# Save predictions
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_name = f"xgb_unseen_predictions_{timestamp}.csv"
results_unseen.to_csv(file_name, index=False)
print(f"Saved: {file_name}")

print(results_unseen.head())
print(
    f"Share predicted positive (1) on unseen: "
    f"{results_unseen['xgb_pred'].mean():.3%} (n={len(results_unseen)})"
)


Saved: xgb_unseen_predictions_2025-09-10_01-19-32.csv
        Clinic                     Service  AmountCharged CPTCode  ClientID  \
1322275     LO  Methadone Maintenance Week         114.00   H0020      4804   
671232    B42C   Methadone Liquid 10mg/1ml           7.80   S0109      4314   
251499     B41   Methadone Liquid 10mg/1ml           3.12   S0109      1552   
1814187   V10A         Drug Screen BHG REF          49.71   80307     11092   
498310    B42A       Individual Counseling          96.00   H0004      1133   

                                  Payer    Provider      AuthStatus  \
1322275  Passport Health Plan by Molina  1932235363  Not Authorized   
671232               Virginia Medicaid   1932696770             N/A   
251499                    OptimaHealth   1932655313  Not Authorized   
1814187       Colorado Medicaid Access   1952807943  Not Authorized   
498310            Anthem HealthKeepers   1598252330  Not Authorized   

           eligStatus DenialFlag  SameDayCli